# Basic `kosmatau3d` model

Here we will examine what are the basic parameters to set in order to establish a simple three-dimensional model. Since we want this example to be easily-relatable, we will attempt to build the `kosmatau3d` version of the famous Hollenbach and Tielens (1999) PDR structure image.

In [1]:
import matplotlib.pyplot as plt
import numpy as np

from kosmatau3d import models
from pprint import pprint

ImportError: /home/yanitski/anaconda3/lib/python3.9/site-packages/cygrid/cygrid.cpython-39-x86_64-linux-gnu.so: undefined symbol: _ZSt28__throw_bad_array_new_lengthv

## Simple constant-density example

This example assumes each voxel in the seup has the same properties (mass, density, $\tau_\mathrm{FUV}$) and only differ in the amount of far-UV radiation. The attenuation of the radiation is then simple to calculate.

For our fiducial case we will assume a voxel size of 0.1 pc, an ensemble density of $10^4$ $\mathrm{cm}^{-3}$, and thus a mass of $\sim$0.16 $M_\odot$.

### without clumpyness

Here we want a voxel fully-filled with one clump. We can estimate this by specifying a clump mass equal to the ensemble mass.

In [1]:
%matplotlib inline

suggested_calc = True

models.constants.n_sigma = 3

models.constants.dtype = np.float32

# Use these to change the voxel properties, which is necessary in the default mode.
parameters = {
                # Model parameters
                'voxel_size' : 0.1,
                 'molecules' : 'all',
                      'dust' : 'PAH',
            'clumpMassRange' : [-0.8],
           'clumpMassNumber' : [1],
                 'clumpNmax' : [1],
             'velocityRange' : [-5, 5],
            'velocityNumber' : 201,

                # Voxel properties
                  'velocity' : 0,
        'ensembleDispersion' : 1,#1,
              'ensembleMass' : [10**2, 1, 1],#7.15, 5.22#10**1,#0.8,
#               'volumeFactor' : [0.8, 0.2],
           'ensembleDensity' : [10**4, 1191, 1191],#4.13, 4.17#10**5,#4.5,
                       'FUV' : 10**6,#1.13, 1.07#1.7,
                      # 'crir' : 2e-16,
    
                # initialisation
                    'dilled' : True,

                # calculation
            'suggested_calc' : suggested_calc,
       'velocity_resolution' : 3,
    
                # logging
                   'verbose' : False,
                     'timed' : False,
                     'debug' : False
          }

t0 = time()
    
vox = models.Voxel()
vox.setProperties(**parameters)
print('initialised:', time()-t1)
vox.calculateEmission(test_calc=test_calc, test_opacity=test_opacity, test_pexp=test_pexp, test_fv=test_fv, verbose=False, timed=False)
print('calculated:', time()-t1)
# vox.plot_clump_number(effective=True)
# vox.plotMolecule(quantity='intensity', kind='slinear', molecule=['C+ 1', 'O 1', 'CO 4', 'CO 45'], moleculeName=[r'$\mathrm{C}^+ 1 \rightarrow 0$', r'$\mathrm{O} 1 \rightarrow 0$', r'$\mathrm{CO} 4 \rightarrow 3$', r'$\mathrm{CO} 45 \rightarrow 46$'])
vox.plotMolecule(quantity='intensity', kind='slinear', molecule=['C+ 1', 'C 1', 'CO 1', 'CO 5'], moleculeName=[r'$\mathrm{C}^+ 1 \rightarrow 0$', r'$\mathrm{C} 1 \rightarrow 0$', r'$\mathrm{CO} 1 \rightarrow 0$', r'$\mathrm{CO} 5 \rightarrow 4$'])
# vox.plotSpectrum(quantity='intensity', kind='slinear', integrated=True)

print(time()-t0, '\n')
print('\nVolume Filling Factor')
print('---------------------')
print('\n'.join(['Clump {}: {:.3f}'.format(_+1, vox.getVolumeFillingFactor()[_]) for _ in range(models.constants.ensembles)]))
print('\nDensity')
print('---------------------')
print('\n'.join(['Clump {}: {:.3e}'.format(_+1, vox.getDensity()[_]) for _ in range(models.constants.ensembles)]))
print()

NameError: name 'models' is not defined

### using a clump distribution

Now we can use a proper distribution of clump masses in each voxel.

## grid files

Every model must be created from some datafiles, at least to obtain the necessary parameters (mass, density, and far-UV radiation). Our purpose does not require anything complex with varying mass or density, but rather we will assume some constant far-UV radiation from one direction and calculate how the absorption and radiative transfer affects the intensity in each voxel. For that reason we will first create mock grid files in a new directory, then `kosmatau3d` will read the model information from there.

In this example we will use voxels of size 1 pc containing mass $100$ $M_\odot$ and density $10^4$ cm$^{-3}$. Each voxel will also need some velocity information, so we want $v_\mathrm{vox}=0$ km s$^{-1}$ and $\sigma_\mathrm{ens}=1$ km s$^{-1}$. Finally the far-UV radiation (neglecting absorption) will be constant at $10^5$ $\chi_D$. We will see later how accounting for absorption will affect the far-UV penetration.

In [8]:
import numpy as np
from scipy.interpolate import interp1d

x = np.linspace(1, 10, num=10)
y = np.ones_like(x)

fx = interp1d(x, y, fill_value='extrapolate')
fx(-5000)

array(1.)

## setting the model parameters

Now that we have grid files from which `kosmatau3d` will read the voxel properties, we simply need to pass the necessary parameters to the `Model` constructor.

In [ ]:
parameters = {
              # Model information
              'history_path': '',# r'/mnt/hpc_backup/yanitski/projects/pdr/KT3_history',
              'directory': '',
              'folder': 'HT99',
              'x': 10,
              'y': 5,
              'z': 0,
              'modelType': 'block',

              # Model parameters
              'resolution': 1,
              'molecules': 'all',
              # 'molecules': ['C+ 1', 
              #               'C 1', 'C 2', 
              #               'CO 1', 'CO 2', 'CO 3', 'CO 4', 'CO 5', 
              #               'CO 6', 'CO 7', 'CO 8', 
              #               '13C+ 1', 
              #               '13C 1', '13C 2', 
              #               '13CO 1', '13CO 2', '13CO 3', '13CO 4', '13CO 5', 
              #               '13CO 6', '13CO 7', '13CO 8', 
              #               'HCO+ 1', 'HCO+ 2', 'HCO+ 3', 'HCO+ 4', 'HCO+ 5'],
              'dust': 'PAH',
              # 'dust': ['240um', '550um'],
              'clumpMassRange': [[-1, 2]],
              'clumpMassNumber': [4],
              'clumpNmax': [None],
              'clumpLogFUV' : None,
              'interclumpLogFUV' : None,
              'velocityRange': [-10, 10],
              'velocityNumber': 21,

              # Flags
              'suggested_calc': True,
              'dilled': True,

              # Property factors
              'hi_mass_factor': 1,
              'h2_mass_factor': 1,
              'ensemble_mass_factor': [1, 1],
              'fuv_factor': 1,
              'density_factor': 1,
              'interclump_hi_ratio': 1,
              'r_cmz': 0,

              # Logging
              'timed': False,
              'debug': False,
              'verbose': False
              }